<a href="https://colab.research.google.com/github/ruman-shaikh/NLP_Project_Grp_1/blob/master/Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%pwd

Mounted at /gdrive


'/content'